<a href="https://colab.research.google.com/github/EDFIAP21091974/RM561352---RM564440---RM566069---RM-566336-A/blob/main/Tarefa2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Célula de configuração para Google Colab
from google.colab import drive
from pathlib import Path
import os

# 1. Montar o Google Drive
drive.mount('/content/drive')

# 2. Definir o caminho base NO SEU DRIVE
BASE_PATH = Path('/content/drive/MyDrive/FarmTech_YOLO_Project/')

# 3. Criar a pasta do projeto no Drive (se ainda não existir)
BASE_PATH.mkdir(parents=True, exist_ok=True)

print(f"✅ Drive montado e BASE_PATH definido como: {BASE_PATH}")

# Navegar para o diretório de trabalho do Colab para clonar os repositórios
# Isso é mais rápido do que clonar diretamente no Drive
os.chdir('/content/')

# Clone o repositório do YOLOv3 (se ainda não tiver clonado nesta sessão)
if not Path('yolov3').exists():
    !git clone https://github.com/ultralytics/yolov3
    !cd yolov3 && pip install -r requirements.txt

# Clone o repositório do YOLOv5 (se ainda não tiver clonado nesta sessão)
if not Path('yolov5').exists():
    !git clone https://github.com/ultralytics/yolov5
    !cd yolov5 && pip install -r requirements.txt

Mounted at /content/drive
✅ Drive montado e BASE_PATH definido como: /content/drive/MyDrive/FarmTech_YOLO_Project
Cloning into 'yolov3'...
remote: Enumerating objects: 12362, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 12362 (delta 78), reused 48 (delta 48), pack-reused 12247 (from 5)
Receiving objects: 100% (12362/12362), 10.84 MiB | 23.83 MiB/s, done.
Resolving deltas: 100% (8348/8348), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.7 MB/s eta 0:00:00
Cloning into 'yolov5'...
remote: Enumerating objects: 17611, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 17611 (delta 8), reused 2 (delta 2), pack-reused 17595 (from 3)
Receiving objects: 100% (17611/17611), 16.84 MiB | 24.35 MiB/s, done.
Resolving deltas: 100% (11984/11984), done.


In [2]:
# ===================================================================
# CÉLULA 1: CONFIGURAÇÃO E IMPORTS (VERSÃO GOOGLE COLAB)
# ===================================================================
import os
import shutil
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from pathlib import Path
from tqdm import tqdm
from google.colab import drive

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# --- Montar Drive e Definir Caminho Base (ESSENCIAL) ---
drive.mount('/content/drive')
BASE_PATH = Path('/content/drive/MyDrive/FarmTech_YOLO_Project/')
BASE_PATH.mkdir(parents=True, exist_ok=True) # Garante que a pasta existe
print(f"📁 Pasta do projeto no Drive: {BASE_PATH}")


# ===================================================================
# CÉLULA 2: PREPARAÇÃO DO DATASET DE CLASSIFICAÇÃO (sem alterações)
# ===================================================================
originais_path = BASE_PATH / 'imagens_originais'
classificacao_path = BASE_PATH / 'dataset_classificacao'
classes = ['cachorros', 'gatos']

def organizar_dataset_classificacao(origem, destino, split_ratio=0.8):
    if destino.exists():
        print(f"⚠️  A pasta '{destino.name}' já existe. Nenhuma ação foi tomada.")
        return
    print(" reorganizando dataset para classificação...")
    for split in ['train', 'val']:
        for classe in classes:
            (destino / split / classe).mkdir(parents=True, exist_ok=True)
    for classe in classes:
        imagens = list((origem / classe).glob('*'))
        random.shuffle(imagens)
        split_point = int(len(imagens) * split_ratio)
        train_files, val_files = imagens[:split_point], imagens[split_point:]
        for f in train_files:
            shutil.copy(f, destino / 'train' / classe / f.name)
        for f in val_files:
            shutil.copy(f, destino / 'val' / classe / f.name)
    print(f"✅ Dataset de classificação criado com sucesso em: {destino}")

organizar_dataset_classificacao(originais_path, classificacao_path)


# ===================================================================
# CÉLULA 3: DEFINIÇÃO DO MODELO CNN (sem alterações)
# ===================================================================
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, 1), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, 3, 1), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 3, 1), nn.ReLU(), nn.MaxPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 6 * 6, 512), # Ajustado para entrada 64x64 com padding 0
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    def forward(self, x):
        return self.classifier(self.features(x))


# ===================================================================
# CÉLULA 4: TREINAMENTO DO MODELO (com pequeno ajuste)
# ===================================================================
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((64, 64)), transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2), transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((64, 64)), transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: datasets.ImageFolder(classificacao_path / x, data_transforms[x]) for x in ['train', 'val']}

# --- AJUSTE PARA COLAB ---
# num_workers=2 é mais estável no Google Colab do que 4
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=2) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# (O resto do loop de treinamento continua exatamente igual...)

# Loop de Treinamento
num_epochs = 25
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}'); print('-' * 10)
    for phase in ['train', 'val']:
        model.train() if phase == 'train' else model.eval()
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in tqdm(dataloaders[phase], desc=f"{phase.capitalize()}"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs); _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                if phase == 'train':
                    loss.backward(); optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
        print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}\n')

print('✅ Treinamento da CNN concluído!')
torch.save(model.state_dict(), BASE_PATH / 'models' / 'cnn_classificador_final.pth')
print("Modelo salvo em 'models/cnn_classificador_final.pth'")

PyTorch version: 2.8.0+cu126
CUDA available: False
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📁 Pasta do projeto no Drive: /content/drive/MyDrive/FarmTech_YOLO_Project
 reorganizando dataset para classificação...
✅ Dataset de classificação criado com sucesso em: /content/drive/MyDrive/FarmTech_YOLO_Project/dataset_classificacao


FileNotFoundError: Found no valid file for the classes cachorros, gatos. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [ ]:
def organizar_dataset_completo(origem, destino, splits=(0.7, 0.15, 0.15)):
    """
    Organiza o dataset em conjuntos de treino, validação e teste.
    """
    if sum(splits) != 1.0:
        print("Erro: A soma das divisões deve ser 1.0")
        return

    if destino.exists():
        print(f"⚠️  A pasta '{destino.name}' já existe. Apague-a primeiro para recriar.")
        return

    print(" Reorganizando dataset em treino, validação e teste...")

    extensoes_validas = ['.jpg', '.jpeg', '.png', '.jfif', '.webp']

    # Criar a nova estrutura de pastas
    for split in ['train', 'val', 'test']:
        for classe in classes:
            (destino / split / classe).mkdir(parents=True, exist_ok=True)

    for classe in classes:
        imagens = []
        for ext in extensoes_validas:
            imagens.extend(list((origem / classe).glob(f"*{ext.lower()}")))
            imagens.extend(list((origem / classe).glob(f"*{ext.upper()}")))

        imagens = list(set(imagens))
        random.shuffle(imagens)

        # Pontos de corte
        ponto1 = int(len(imagens) * splits[0])
        ponto2 = int(len(imagens) * (splits[0] + splits[1]))

        # Dividir arquivos
        train_files = imagens[:ponto1]
        val_files = imagens[ponto1:ponto2]
        test_files = imagens[ponto2:]

        # Copiar arquivos
        for f in train_files: shutil.copy(f, destino / 'train' / classe / f.name)
        for f in val_files: shutil.copy(f, destino / 'val' / classe / f.name)
        for f in test_files: shutil.copy(f, destino / 'test' / classe / f.name)

    print(f"✅ Dataset completo criado com sucesso em: {destino}")

originais_path = BASE_PATH / 'imagens_originais'
classificacao_path = BASE_PATH / 'dataset_classificacao'
classes = ['cachorros', 'gatos']

organizar_dataset_completo(originais_path, classificacao_path)

In [ ]:
# CÉLULA DE TESTE FINAL DO MODELO

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

# ... (código para carregar dados e o modelo - sem alterações) ...
# 1. Carregar os dados de Teste
test_dir = classificacao_path / 'test'
test_transforms = transforms.Compose([
    transforms.Resize((64, 64)), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_dataset = datasets.ImageFolder(test_dir, test_transforms)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

# 2. Carregar o Modelo Treinado
caminho_modelo_salvo = BASE_PATH / 'models' / 'cnn_classificador_final.pth'
model = SimpleCNN(num_classes=2).to(device)
model.load_state_dict(torch.load(caminho_modelo_salvo))
model.eval()

# 3. Fazer as Previsões
todas_as_preds = []
todas_as_labels = []
with torch.no_grad():
    for inputs, labels in tqdm(test_dataloader, desc="Testando..."):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        todas_as_preds.extend(preds.cpu().numpy())
        todas_as_labels.extend(labels.cpu().numpy())

# 4. Gerar e Exibir os Resultados
print("\n" + "="*50); print("📊 RELATÓRIO DE CLASSIFICAÇÃO NO CONJUNTO DE TESTE"); print("="*50)
nomes_classes = image_datasets['train'].classes
report = classification_report(todas_as_labels, todas_as_preds, target_names=nomes_classes)
print(report)

# Salvar o relatório em um arquivo de texto ---
report_path = BASE_PATH / 'results' / 'cnn_classification_report.txt'
with open(report_path, 'w') as f:
    f.write(report)
print(f"\n✅ Relatório de classificação salvo em: {report_path}")
# ----------------------------------------------------

print("\n" + "="*50); print("🔲 MATRIZ DE CONFUSÃO"); print("="*50)
fig, ax = plt.subplots(figsize=(8, 6))
ConfusionMatrixDisplay.from_predictions(todas_as_labels, todas_as_preds, ax=ax, display_labels=nomes_classes, cmap='Blues')
plt.title('Matriz de Confusão no Conjunto de Teste')

# Salvar o gráfico da matriz de confusão ---
figure_path = BASE_PATH / 'results' / 'cnn_confusion_matrix.png'
plt.savefig(figure_path, dpi=150, bbox_inches='tight')
print(f"✅ Matriz de confusão salva em: {figure_path}")
# ---------------------------------------------------

plt.show()